#### CASCODE COMMON-SOURCE (11)
### Common source amplifier with cascode transistor or cascode current mirror

* The cascode transistor M2 for the common source amplifier stages M1 is an impedance converter. 
  * The input impedance of the cascode transitor M2 is low $\frac{1}{g_m}$ while the output impedance is large ${g_m}{r_{cascode}}{r_o}$
  * The M1 is sized for the required $g_m$, $r_o$ and the M2 is sized for overall desired $r_{cascode}$ 
  * The operating points are chosen as per the desired bias current in the branch which could be set by another active load device not shown here. The biasing of the cascode device could be generated by another bias voltage source, but for the purpose of the block , it is defined at a level of $2*V_{DSAT}+V_{THN}$ of about 1V. Hence, specifications: 
    * $V_{BIAS}$ = 1V
    * $I_{BIAS}$ = 1uA
    * $g_{m, M1}$ = 10mS
    * $r_{o, M1}$ = $\frac{1}{g_{ds, M1}}$ = 100KOhms
    * $r_{cascode, M2}$ = $\frac{1}{g_{ds, M2}}$ = 50KOhms

  * To verify, overall small signal output impedance is ${10{e}^{-3}}*{100e^3}*{5e^3}$ = $5e^7$ Ohms. 

In [2]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

In [3]:
nfet_01v8_lvt = lk('../techsweep/simulation/nfet_01v8_lvt.mat')

#### Specifications

In [4]:
gm = 10e-3
ro = 100e3
rcascode = 50e3

#### Design Choices

In [32]:
l = 0.15
# Large gmid for input device for higher gain in weak inversion
# & small gmid for cascode device to keep in deep saturation
gm_id_in = np.array([20, 10])
gm_id_casc = np.array([6, 15])

#### Sizing and Benchmarking

In [38]:
# Sizing the input active device for amplications
id = gm/gm_id_in
# Look up JD for the requried GM_ID and L.
jd = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_id_in, L=l)
# Get W from the ID and JD
w = id/jd
# Get the gds of the input device from looking up gmid for the length and then multiplying it with W.
gds_in = nfet_01v8_lvt.lookup('GDS_W', GM_ID=gm_id_in, L=l)*w
## The output impedance is inverse of the conductance.
rout_in = 1/gds_in
# print(f" Checking the availability of the data from datafame: id", id, jd, w)
# Printing for one column
# df = pd.DataFrame([gm_id_in, id, jd],['gm_id', 'id', 'jd'], columns=['option1'])
df = pd.DataFrame([gm_id_in, id, jd, w, gds_in, rout_in],
                    ['gm_id', 'id', 'jd', 'w', 'gds', 'ROUT'], 
                        columns=[f'Option1 : l = {l}', f'Option2: l = {l}'])
df

,Option1 : l = 0.15,Option2: l = 0.15
gm_id,20.000000,10.000000
id,0.000500,0.001000
jd,0.000003,0.000032
w,162.475199,30.942530
gds,0.000565,0.000599
ROUT,1770.212256,1670.351713


#### Spice Netlist

In [39]:
%%writefile ./sizing_cascoded_common_source.spice
** Cascoded common source

.include /foss/pdks/sky130A/libs.tech/ngspice/corners/tt.spice
.param mc_mm_switch=0
.param lx=0.15 wx=162.5 nfx=40 idx=0.5m
.save @m.M1.msky130_fd_pr__nfet_01v8_lvt
.save @m.M2.msky130_fd_pr__nfet_01v8_lvt

M2 d g s 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
M1 d g s 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
vg  g  0  1
vd  d  0  1
is  s  0  {2*idx}

.control
  op
  *show
  print @m.M1.msky130_fd_pr__nfet_01v8_lvt[gm]
  print @m.M1.msky130_fd_pr__nfet_01v8_lvt[id]
  print @m.M1.msky130_fd_pr__nfet_01v8_lvt[gds]
.endc
.end

Writing ./sizing_cascoded_common_source.spice
